In [1]:
from selenium.common.exceptions import (NoSuchElementException, ElementClickInterceptedException,
WebDriverException, StaleElementReferenceException)
from selenium import webdriver

import time
import pandas as pd

In [2]:
path = '/usr/bin/geckodriver'

In [3]:
def get_jobs(keyword, num_jobs, path, sleep_time):
    
    '''
    Gathers jobs as a dataframe, scraped from Glassdoor
    '''

    options = webdriver.FirefoxOptions()
    
    driver = webdriver.Firefox(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?&sc.keyword=" ' + keyword
    
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  
        
        time.sleep(sleep_time)

        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass
        except NoSuchElementException:
            pass
        except StaleElementReferenceException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_id("prefix__icon-close-1").click()
        except NoSuchElementException:
            pass
        except StaleElementReferenceException:
            pass

        job_buttons = driver.find_elements_by_class_name("jl")
        for job_button in job_buttons:  
    
            if len(jobs) != 0:
                print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
            
            try:
                job_button.click()
                time.sleep(1.5)
                collected_successfully = False
            except StaleElementReferenceException:
                pass
            
            while not collected_successfully:
                try:
                    try:
                        company = driver.find_element_by_xpath('.//div[@class = "css-87uc0g e1tk4kwz1"]').text
                    except NoSuchElementException:
                        company = -1
                    try:
                        location = driver.find_element_by_xpath('.//div[@class = "css-56kyx5 e1tk4kwz5"]').text
                    except NoSuchElementException:
                        location = -1
                    try:
                        title = driver.find_element_by_xpath('.//div[@class = "css-1vg6q84 e1tk4kwz4"]').text
                    except NoSuchElementException:
                        title = -1
                    try:
                        rating = driver.find_element_by_xpath('.//span[@class = "css-1m5m32b e1tk4kwz2"]').text
                    except NoSuchElementException:
                        rating = -1
                    try:
                        salary = driver.find_element_by_xpath('.//span[@class = "css-56kyx5 css-16kxj2j e1wijj242"]').text
                    except NoSuchElementException:
                        salary = -1
                    try:
                        description = driver.find_element_by_xpath('.//div[@class = "jobDescriptionContent desc"]').text
                    except NoSuchElementException:
                        description = -1
                    try:
                        founded = driver.find_element_by_xpath('.//div[@class = "infoEntity"]//label[text() = "Founded"]//following-sibling::*').text
                    except NoSuchElementException:
                        founded = -1
                    try:
                        size = driver.find_element_by_xpath('.//div[@class = "infoEntity"]//label[text() = "Size"]//following-sibling::*').text
                    except NoSuchElementException:
                        size = -1
                    try:
                        industry = driver.find_element_by_xpath('.//div[@class = "infoEntity"]//label[text() = "Industry"]//following-sibling::*').text
                    except NoSuchElementException:
                        industry = -1
                    try:
                        sector = driver.find_element_by_xpath('.//div[@class = "infoEntity"]//label[text() = "Sector"]//following-sibling::*').text
                    except NoSuchElementException:
                        sector = -1
                    try:
                        company_type = driver.find_element_by_xpath('.//div[@class = "infoEntity"]//label[text() = "Type"]//following-sibling::*').text
                    except NoSuchElementException:
                        company_type = -1
                    try:
                        revenue = driver.find_element_by_xpath('.//div[@class = "infoEntity"]//label[text() = "Revenue"]//following-sibling::*').text
                    except NoSuchElementException:
                        revenue = -1
                    collected_successfully = True
                except:
                    time.sleep(1)

            jobs.append({"Company" : company,
                         "Location": location,
                         "Title" : title,
                         "Rating" : rating,
                         "Salary" : salary,
                         "Description" : description,
                         "Founded" : founded,
                         "Size" : size,
                         "Industry" : industry,
                         "Sector" : sector,
                         "Type" : company_type,
                         "Revenue" : revenue})

        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
            try:
                print("I got a page loading error")
                driver.find_element_by_xpath("//button[contains(text(), 'Retry your search')]").click()
            except NoSuchElementException:
                pass
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)

In [4]:
number_of_jobs = int(input("Enter the number of jobs to scrape: "))

start = time.time()

df = get_jobs('data scientist',number_of_jobs,path,1)

end = time.time()

print("Scraping {} jobs took {} seconds".format(number_of_jobs,round((end - start),2)))

Enter the number of jobs to scrape: 10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10
I got a page loading error
Scraping 10 jobs took 64.56 seconds


In [5]:
df.to_csv('glassdoor data scientist salary.csv', index = False)